In [93]:
import os, random, glob, pickle, collections, math
import numpy as np
import pandas as pd
import ujson as json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

In [103]:
TRAIN_DIR = '../data/train/'
CHECKPOINT_DIR = './checkpoints/checkpoint4/'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BatchSize = 128
LearningRate = 1e-4


In [72]:
GT_crop_bboxs_df[4370:4372]

,image_class,image_file,crop_index,crop_class,x_min,y_min,x_max,ymax
4370,YFT,img_07911.jpg,1.0,YFT,724.290780,94.680851,1168.617021,347.872340
4371,ALB,img_00003.jpg,0.0,NoF,299.452861,26.285698,1168.160268,286.172768


In [ ]:
def preprocess_input(x):
    #resnet50 image preprocessing
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

train_datagen = ImageDataGenerator(
    preprocessing_function = pre_fn
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

In [104]:
FISH_CLASSES.index('NoF')

0

In [ ]:
nbr_perClass = int(BatchSize / len(FISH_CLASSES))

    with self.lock:
    index_array, current_index, current_batch_size = next(self.index_generator)
    # The transformation of images is not under thread lock
    # so it can be done in parallel
    batch_x = np.zeros((BatchSize, ROWS, COLS, 3), dtype=K.floatx())
    batch_y = np.zeros((BatchSize, len(FISH_CLASSES)), dtype=K.floatx())
    fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nbr_perClass, replace=False),:]
    batch_df = GT_crop_bboxs_df.groupby('crop_class', as_index=True).apply(fn)
    i = 0
    for index,row in batch_df.iterrows():
        row = row.tolist()
        image_file = os.path.join(row[0], row[1])
        fish = row[3]
        bbox = row[4:8]
        img = image.load_img(TRAIN_DIR+image_file,target_size=(ROWS,COLS))
        x = img_to_array(img, dim_ordering=self.dim_ordering)
        x = train_datagen.random_transform(x)
        x = train_datagen.standardize(x)
        batch_x[i] = x
        batch_y[i,FISH_CLASSES.index(fish)] = 1
        i += 1
    return batch_x, batch_y
    
def generate_arrays_from_file(path):
    while 1:
        f = open(path)
        for line in f:
            # create numpy arrays of input data
            # and labels, from each line in the file
            x, y = process_line(line)
            img = load_images(x)
            yield (img, y)
        f.close()

    while 1:
        if seed is not None:
            np.random.seed(seed + self.total_batches_seen)
        if self.batch_index == 0:
            index_array = np.arange(n)
            if shuffle:
                index_array = np.random.permutation(n)

        current_index = (self.batch_index * batch_size) % n
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            self.batch_index += 1
        else:
            current_batch_size = n - current_index
            self.batch_index = 0
        self.total_batches_seen += 1
        yield (index_array[current_index: current_index + current_batch_size],
               current_index, current_batch_size)
        
       
            
model.fit_generator(generate_arrays_from_file('/my_file.txt'),
        samples_per_epoch=10000, nb_epoch=10)